In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade -q gspread

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")
worksheet_cypher_queries = sh.worksheet(title="cypher_queries")

In [4]:
"""
Loop through rows of spreadsheet
Download images
"""
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for row_counter in range(2,254):
    file_num = worksheet_pages.get_values("C"+str(row_counter))[0][0]

In [5]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 60.4 MB/s 
  Created wheel for openai: filename=openai-0.24.0-py3-none-any.whl size=55926 sha256=c42205a794ed8e2ec0fcfa3829be9c02585b35acd9afec9834cba3f901facdcd
  Stored in directory: /root/.cache/pip/wheels/94/2b/ee/7649ac33c142e3fde2081bd8769337b5e75710fd4b885cd2c6
Successfully built openai


In [6]:
settings_path = "/content/drive/MyDrive/william_white/"
    
import os
import json
settings_file = settings_path+'openai.json'
with open(settings_file, "r") as read_file:
    auth_data = json.load(read_file)

os.environ['OPENAI_API_KEY']=auth_data["OPENAI_API_KEY"]

In [ ]:
# This was written by Dr. Shawn Graham - I am adapting this
# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!
# https://medium.com/geekculture/relationship-extraction-with-gpt-3-bb019dcf41e5
# https://github.com/dgg32/gpt-3-extract

# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!

# the wait so that we don't screw up api limits is from the gpt-3 examples repo from openai


import os
import sys
import openai

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


#training_file = '/content/drive/MyDrive/fieldbooks_antioch/shorter-training.txt'

#input_file = '/content/drive/MyDrive/fieldbooks_antioch/sample-test.txt'
training_file = '/content/drive/MyDrive/fieldbooks_antioch/ocr-translation.txt'
input_file = '/content/drive/MyDrive/fieldbooks_antioch/r21.txt'

openai.api_key = os.getenv("OPENAI_API_KEY")

training = "\n".join(open(training_file, 'r').readlines()) + "\n"
text = "\n".join(open(input_file, 'r').readlines()) + "\n"

def extract_relation (text):
  my_prompt = training + text

#  response = openai.Completion.create(
  response = completion_with_backoff(
    engine="text-davinci-002",
    prompt=my_prompt,
    temperature=0.0,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["END"]
  )

  return response.choices[0].text.lstrip()

In [ ]:
with open(input_file, 'r') as file_in:
    for line in file_in:
        if len(line.strip()) > 0:
          res = extract_relation(line.strip() + "\n")

          print(res)

In [ ]:
    training_text_french = """Correct the OCR errors in this French text. Print the correction underneath a subheading called 'CORRECTION'. Then, underneath a subheading called 'TRANSLATION', translate into English:

C'est un terr4in situé à l'Est or la Route ; il est conlyn
l'un 1 derrières massimo ; plante de quelpas figuiers - dal
acting but jeers . it off presseurs ; in ya reme'de mai's.
Plusieurs tuns voision, d'environ 5m de profondeur, as servi
À carrier ; men a extrait is press dailles, with fragment
gardent belle apparence. Les travaux net avéli pas 11. Pust
De puits voisin A la maison voisine."""

    training_text_english ="""Correct the OCR errors in this English text. Print the correction underneath a subheading called 'CORRECTION':

2nd Period Hippodrome Time of Constantine.
Rubble concrete foundations- with large limestone blocks out on
them in first course of superstructure. Superstar + here of limestone
Blocks for facing which willle concrete cose. For measurements of bloodes of Progressce
Outside west well five subtle foundation up flush to H-S call of
period one (cf photo). Over foren date on is aubbly concrete care of
superstructure much one limestone block in situ."""

In [9]:
# This was written by Dr. Shawn Graham - I am adapting this
# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!
# https://medium.com/geekculture/relationship-extraction-with-gpt-3-bb019dcf41e5
# https://github.com/dgg32/gpt-3-extract

# https://github.com/dgg32/gpt-3-extract
# the original code didn't read in the test text!

# the wait so that we don't screw up api limits is from the gpt-3 examples repo from openai


import os
import sys
import openai

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

openai.api_key = os.getenv("OPENAI_API_KEY")


def compose_my_prompt(input_text, input_text_language = "English"):

    training_text_french = """Correct the OCR errors in this French text. Print the correction underneath a subheading called 'CORRECTION'. Then, underneath a subheading called 'TRANSLATION', translate into English:"""
    training_text_english ="""Correct the OCR errors in this English text:"""

    if input_text_language == "French":
        #training = "\n".join(training_text_french) + "\n"
        training = training_text_french + "\n"
    else:
        #training = "\n".join(training_text_english) + "\n"
        training = training_text_english + "\n"

    #text = "\n".join(input_text) + "\n"
    text = input_text + "\n"    
    return training + text

def compose_my_prompt_cypher_query(input_text):
    # I assume all text is English
    training_text = """Medici started dealing in antiquities in Rome during the 1960s (Silver 2009: 25). In July 1967, he was convicted in Italy of receiving looted artefacts, though in the same year he met and became an important supplier of antiquities to US dealer Robert Hecht (Silver 2009: 27-9). In 1968, Medici opened the gallery Antiquaria Romana in Rome and began to explore business opportunities in Switzerland (Silver 2009: 34). It is widely believed that in December 1971 he bought the illegally-excavated Euphronios (Sarpedon) krater from tombaroli before transporting it to Switzerland and selling it to Hecht (Silver 2009: 50). In September 1995, the Italian Carabinieri, in conjunction with Swiss police, raided Italian antiquities dealer Giacomo Medici’s storerooms in Geneva Freeport. The raids on Medici’s storerooms were part of a wide-ranging investigation into the antiquities trade out of Italy, and more raids followed in 2001 on US dealer Robert Hecht’s apartment in Paris and in 2002 and 2005 on Italian dealer Gianfranco Becchina’s premises in Basel. The evidence collected during the course of the Italian investigation caused several US art museums to return material to Italy, including the Cleveland Museum of Art. As long ago as 1973, Cleveland’s then-curator of ancient art John Cooney had expressed his belief that ninety-five percent of material in the United States had been ‘smuggled in’, adding that even for ‘hot’ objects, he was only concerned to verify their legal entry into the United States, identifying Switzerland as a point of transit (Shirey 1973). Cooney described Hecht as ‘always meticulous and upright’, with ‘one of the best eyes in the business’, adding that he never asked Hecht for documents of provenance—he thought it would be ridiculous (Shirey 1973).
END

OUTPUT:

CREATE (p1:PERSON {personName:'Giacomo Medici',type:'dealer'})
CREATE (p2:PERSON {personName:'Gianfranco Becchina',type:'dealer'})
CREATE (p3:PERSON {personName:'John Cooney',type:'curator'})
CREATE (p4:PERSON {personName:'Robert Hecht',type:'dealer'})
CREATE (o1:ORGANIZATION {orgName:'Italian Carabinieri',type:'police'})
CREATE (o2:ORGANIZATION {orgName:'Cleveland Art Museum',type:'museum'})
CREATE(l1:PLACE{placeName:'Geneva',type:'city'})
CREATE(l2:PLACE{placeName:'Basel',type:'city'})
CREATE(l3:PLACE{placeName:'Italy',type:'country'})
CREATE(l4:PLACE{placeName:'Medici storerooms',type:'crime scene'})

MATCH (a {orgName:'Italian Carabinieri'}), (b {placeName:'Medici storerooms'}) MERGE (a)-[:RAIDED]->(b);
MATCH (a {placeName:'Medici storerooms'}), (b {placeName:'Geneva'}) MERGE (a)-[:LOCATED_IN]->(b);
MATCH (a {orgName:'Italian Carabinieri'}), (b {personName:'Gianfranco Becchina'}) MERGE (a)-[:RAIDED]->(b);
MATCH (a {personName:'Gianfranco Becchina'}), (b {placeName:'Basel'}) MERGE (a)-[:LOCATED_IN]->(b);
MATCH (a {orgName:'Cleveland Art Museum'}), (b {placeName:'Italy'}) MERGE (a)-[:RETURNED_ARTIFACTS_TO]->(b);
MATCH (a {personName:'Giacomo Medici'}), (b {placeName:'Medici storerooms'}) MERGE (a)-[:OWNED]->(b);
MATCH (a {personName:'John Cooney'}), (b {orgName:'Cleveland Art Museum'}) MERGE (a)-[:CURATOR_OF]->(b);
MATCH (a {personName:'Giacomo Medici'}), (b {personName:'Robert Hecht'}) MERGE (a)-[:SOLD_TO]->(b);"""

def compose_my_prompt_cypher_query_shorter(input_text):
    # I assume all text is English
    training_text = """Extract all subjects, objects, and predicates from the article as correctly formatted CREATE and MATCH cypher queries.

I provide some example queries for you to follow. List CREATE statements first, then MATCH statements:

CREATE (medici {personName:'Giacomo Medici',type:'dealer'})
CREATE (hecht {personName:'Robert Hecht',type:'dealer'})
MATCH (a {personName:'Giacomo Medici'}), (b {personName:'Robert Hecht'}) MERGE (a)-[:SOLD_TO]->(b);"""


    #training = "\n".join(training_text_english) + "\n"
    training = training_text + "\n"

    #text = "\n".join(input_text) + "\n"
    text = input_text + "\nEND\n" 
    return training + text

def extract_relation (my_prompt):
    response = completion_with_backoff(
        engine="text-davinci-002",
        prompt=my_prompt,
        temperature=0.0,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["END"]
        )
    return response.choices[0].text.lstrip()

In [ ]:
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for sheet_row in range(12,254):
    num_pages = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
    row_language = worksheet_pages.get_values("W"+str(sheet_row))[0][0]
    for page_num in range(1,num_pages + 1):
        if page_num == 1:
            page_col = "I"
            transcribed_orig = "Q"
            transcribed_contrast = "R"
            openai_text_col = "X"
        else:
            if page_num == 2:
                page_col = "K"
                transcribed_orig = "S"
                transcribed_contrast = "T"
                openai_text_col = "Y"
        page_file_name = worksheet_pages.get_values(page_col + str(sheet_row))[0][0]
        if len(page_file_name.strip()) > 0:
            print("row",sheet_row, page_file_name, "page", page_num, "of", num_pages)
        try:    
            transcribed_orig_text = worksheet_pages.get_values(transcribed_orig + str(sheet_row))[0][0]
        except:
            transcribed_orig_text = ""          
        try:
            transcribed_contrast_text = worksheet_pages.get_values(transcribed_contrast + str(sheet_row))[0][0]
        except:
            transcribed_contrast_text = ""      
        if len(transcribed_orig_text) > len(transcribed_contrast_text):
            transcribed_text = transcribed_orig_text
        else:
            transcribed_text = transcribed_contrast_text
        my_prompt = compose_my_prompt(transcribed_text, row_language)
        print("my_prompt:",len(my_prompt))
        openai_text = extract_relation(my_prompt)
        worksheet_pages.update(openai_text_col +
                       str(sheet_row) +
                       ':' + openai_text_col +
                       str(sheet_row),
                       [[openai_text]]) 

row 12 ANT_FB_1932-002-0010_p1.jpg page 1 of 2
my_prompt: 247
row 12 ANT_FB_1932-002-0010_p2.jpg page 2 of 2
my_prompt: 1252
row 13 ANT_FB_1932-002-0011_p1.jpg page 1 of 2
my_prompt: 117
row 13 ANT_FB_1932-002-0011_p2.jpg page 2 of 2
my_prompt: 46
row 14 ANT_FB_1932-002-0012_p1.jpg page 1 of 1
my_prompt: 1337
row 15 ANT_FB_1932-002-0013_p1.jpg page 1 of 2
my_prompt: 1407
row 15 ANT_FB_1932-002-0013_p2.jpg page 2 of 2
my_prompt: 206
row 16 ANT_FB_1932-002-0014_p1.jpg page 1 of 1
my_prompt: 948
row 17 ANT_FB_1932-002-0015_p1.jpg page 1 of 2
my_prompt: 1196
row 17 ANT_FB_1932-002-0015_p2.jpg page 2 of 2
my_prompt: 1247
row 18 ANT_FB_1932-002-0016_p1.jpg page 1 of 1
my_prompt: 175
row 19 ANT_FB_1932-003-0000_p1.jpg page 1 of 1
my_prompt: 954
row 20 ANT_FB_1932-003-0001_p1.jpg page 1 of 1
my_prompt: 226
row 21 ANT_FB_1932-003-0002_p1.jpg page 1 of 1
my_prompt: 549
row 22 ANT_FB_1932-003-0003_p1.jpg page 1 of 1
my_prompt: 249
row 23 ANT_FB_1932-003-0004_p1.jpg page 1 of 1
my_prompt: 258
row 

In [11]:
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for sheet_row in range(2,254):
    num_pages = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
    row_language = worksheet_pages.get_values("W"+str(sheet_row))[0][0]
    for page_num in range(1,num_pages + 1):
        if page_num == 1:
            page_col = "A"
            openai_text_col = "C"
            cypher_query_col = "E"
        else:
            if page_num == 2:
                page_col = "B"
                openai_text_col = "D"
                cypher_query_col = "F"
        page_file_name = worksheet_cypher_queries.get_values(page_col + str(sheet_row))[0][0]
        if len(page_file_name.strip()) > 0:
            print("row",sheet_row, page_file_name, "page", page_num, "of", num_pages)
        try:    
            transcribed_text = worksheet_cypher_queries.get_values(openai_text_col + str(sheet_row))[0][0]
        except:
            transcribed_text = ""          

        my_prompt = compose_my_prompt_cypher_query_shorter(transcribed_text)
        print("my_prompt's length:",len(my_prompt),"prompt: (below)")
        print(my_prompt)
        openai_text = extract_relation(my_prompt)
        print("==================================")
        print(openai_text)
        print("==================================")
        worksheet_cypher_queries.update(cypher_query_col +
                       str(sheet_row) +
                       ':' + cypher_query_col +
                       str(sheet_row),
                       [[openai_text]]) 

Streaming output truncated to the last 5000 lines.
CREATE (naise_pool {name:'Naise f pool',type:'pool'})
CREATE (octagon {name:'the octagon',type:'octagon'})
CREATE (apse {name:'W. apse',type:'apse'})
CREATE (dome_rousson {name:'W. also the dome rousson',type:'dome'})
CREATE (c_rom_s {name:'C. Rom S.',type:'shore'})
CREATE (second_course {name:'the second course',type:'course'})
CREATE (jest_sw_dess {name:'Jest the Sw dess',type:'thing'})
MATCH (a {name:'Naise f pool'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
MATCH (a {name:'W. apse'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
MATCH (a {name:'W. also the dome rousson'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
MATCH (a {name:'C. Rom S.'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
MATCH (a {name:'the second course'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
MATCH (a {name:'Jest the Sw dess'}), (b {name:'the octagon'}) MERGE (a)-[:ENCLOSES]->(b);
row 186 ANT_FB_1932-004-0144

In [ ]:
%%html
<!-- Thanks to https://gist.github.com/bollwyvl/e51b4e724f0b82669c84 -->
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/mermaid/9.1.7/mermaid.min.js" integrity="sha512-1ypa9tdUrJAWv5g28Mb5x0zXaUuI4SBofKff88OGyk5D/oOd4x1IPxYHsx3K81bwBKt8NVUvGgw7TgNZ6PJX2A==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
<style>
    .mermaid *{font-family: sans-serif; }
    .mermaid .node, .mermaid .cluster{
      fill: white !important;
      stroke: black !important;
      stroke-width: 1px !important;
    }
    .mermaid div{
      text-align: center;
    }
    .mermaid .label{
      color: black;
    }
</style>
<script>$(function(){
    // mermaid load a touch weirdly: try immediately, but try again later if it's not available
    var initMermaid = function(){
        return (window.mermaid && mermaid.init()) || setTimeout(initMermaid, 50);         
    }
    initMermaid();

    // for live editing, re-render only the text of the current cell
    window.IPython && $(IPython.events).on("rendered.MarkdownCell", function(evt, data){
        // this is using a hacked mermaid that accepts some nodes!
        mermaid.init(undefined, data.cell.element.find(".mermaid"));
    });
});</script>
<h2>Content pipeline</h2>
<div class="mermaid">
graph TD
P1["OCR original page with Azure Cognitive Services."]
P1-->P2["OCR contrasted page with Azure Cognitive Services."]
P2-->P3{"Is OCR of original longer/larger?"}
P3-->|Yes|P4["Use OCR of original"]
P3-->|No|P5["Use OCR of contrasted"]
P4-->P6{"English or French?"}
P5-->P6
P6-->|French|P7["Correct OCR, Translate"]
P6-->|English|P8["Correct OCR"]
P8-->P9["Save text in spreadsheet."]
P7-->P9
P9-->P10["Openai create knowledge graph cypher queries"]
</div>

# scratch pads

In [ ]:
import openai
response = openai.Edit.create(
  model="text-davinci-edit-001",
  input="""3 Juin
# 15h. Toynus pes trace de mouse de travail l'avance
palièrement. La jour Est abe tranchée svangereure .
I y avait une poche de fiers ; les précautions ont pris
pom éviter is éboulements . Le travail ost of anise avec
parcours coventaire par deux camps
In trouve pasmi b débits un fragment de chapation ()
16
5
1
""",
  instruction="What language is this?",
  temperature=0.7,
  top_p=1
)
#print(response)
print(response.choices[0].text.lstrip())

3 Juin
# 15h. Toynus pes trace de mouse de travail l'avance palièrement. La jour Est abe tranchée svangereure. I y avait une poche de fiers; les précautions ont pris pom éviter is éboulements. Le travail ost of anise avec parcours coventaire par deux camps In trouve pasmi b débits un fragment de chapation () 16 5 1



In [ ]:
response = openai.Edit.create(
  model="text-davinci-edit-001",
  input=""""
hours numming ExW. House wall ar 0.20 helaso surface ) en oito
ovulation wall of hojeps drove wa
· mostly destroyed, alepch of foundation wall. here are. 1.40 de.
Inscription un house et NW corner.
TH Z TIT
KCKOY
Jreal French Hippoknow -A - at 4ch on east side - L3 ml
Irgend sur limite ce: 6. m ready what face + Sim. on Stuck
20 april
N. S.
21 april
Come - bate Roman mostly a late arabic
Trench summing E-W or west side of Supportcome - late a which
wall at wetallend of thand QQ25 m.
Black loom - lenge straks - we pott eng lule buck , file
22 april
Sonic Rap- dia. 0.325 homestore
0~ - 0,35
mutilated
"92
"
""",
  instruction="Fix OCR errors.",
  temperature=0.4,
  top_p=1
)
#print(response)
print(response.choices[0].text.lstrip())

"
hours numming ExW. House wall ar 0.20 helaso surface ) en oito ovulation wall of hojeps drove wa · mostly destroyed, alepch of foundation wall. here are. 1.40 de.
Inscription un house et NW corner. TH Z TIT KCKOY Jreal French Hippoknow -A - at 4ch on east side - L3 ml Irgend sur limite ce: 6. m ready what face + Sim. on Stuck 20 april N. S. 21 april Come - bate Roman mostly a late arabic Trench summing E-W or west side of Supportcome - late a which wall at wetallend of thand QQ25 m. Black loom - lenge straks - we pott eng lule buck , file 22 april Sonic Rap- dia. 0.325 homestore 0~ - 0,35 mutilated "92 "



In [ ]:
            response = openai.Completion.create(
            model="text-davinci-002",
            prompt=generate_prompt(animal),
            temperature=0.6,
        )
        return redirect(url_for("index", result=response.choices[0].text))